# Example Extension Method: Witness to Vietoris Rips
* This notebook shows an application of the <b>bar-to-bars extension method</b> to compare barcodes obtained from a Witness filtration to a Vietoris-Rips filtration.
* <b> Implementation </b>: Our function implements a component-wise bar-to-bars extension method with $\mathbb{F}_2$ coefficients. It assumes that all bars of barcodes have unique death times. 
* <b> Comparing W to VR </b>: Given point clouds `P` and `Q`, let `D_VR` be the distance matrix on `P`, and let `D_W` be the cross-distance matrix between `P` and `Q`. Given a selected bar in the Witness barcode, the extension method finds all representations in the Vietoris-Rips barcode. Using paper notations, the Witness filtration $W^\bullet$ corresponds to filtration $Z^\bullet$ and the VR filtration $VR^{\bullet}$ corresponds to filtration $Y^\bullet$. 
* <b> Example data</b>: 
    * `P`: 300 points sampled from a square torus
    * `Q`: 20 points sampled along the S1 of the torus.
    * The Vietoris-Rips filtration will be built on `P`.
    * The Witness filtration will use `P` as a landmark and `Q` as witnesses. 
* <b> Contents </b>
    1. Load points and visualize
    2. Examine the two barcodes
    3. Apply bar-to-bars extension method
    4. Explore cycle extension & bar extension under fixed interval decomposition of `barcode(Y)`.
    5. Explore alternative bar extensions under all possible interval decompositions of `barcode(Y)`.

In [1]:
using Revise
includet("../extension_method.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15779328884790373707\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16013462558200971611\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15779328884790373707\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


In [2]:
using .ext
using DelimitedFiles
using Distances
using Eirene
using HDF5
using Printf
using JLD
using Plots

# 1. load points and visualize
* `P`: 300 points sampled from a square torus.
* `Q`: 20 points along one of the S1's of the torus. Used as a Witness when constructing the Witness filtration.
* `D_VR`: Distance among points `P`. Used to construct the VR filtration on `P`.
* `D_W`: Cross-system distance between `P` and `Q`. Used to construct the Witness filtration.  

Load data

In [3]:
# directory
directory = "data/torus_circle/"

# load points
P = h5read(directory * "coords.h5", "torus")
Q = h5read(directory * "coords.h5", "circle")

# print number of points 
print("number of points in P: ", size(P,1), "\n")
print("number of points in Q: ", size(Q,1))

# load distance matrices
D_VR = readdlm(directory * "distance_torus.csv")
D_W = readdlm(directory * "distance_torus_circle.csv");

number of points in P: 300
number of points in Q: 20

Plot the points on square torus. Assume that the sides of the squares have been identified to represent a torus

In [4]:
p = plot_P_Q(P, Q)

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Examine the Witness and Vietoris-Rips barcodes

Run Eirene on Witness and VR filtration

In [5]:
# Witness filtration
W = compute_Witness_persistence(D_W, maxdim = 1);

# VR filtration
C_VR = eirene(D_VR);

Plot the two barcodes

In [6]:
dim = 1

# get barcodes
barcode_W = barcode(W["eirene_output"], dim = dim)
barcode_VR = barcode(C_VR, dim = dim)

# plot the VR barcode and the Witness barcode
p1 = plot_barcode(barcode_W, title = "Barcode(W)", lw = 5)
p2 = plot_barcode(barcode_VR, title = "Barcode(VR)", lw = 2)
plot(p1, p2, layout = grid(2,1), size = (500, 500))

<!DOCTYPE html>
 
 
 Plots.jl

# 3. Apply bar-to-bars extension method

<b> Hover over the above barcode to select bar of interest. </b>
* For example, we selected bar 2 in `barcode(W)`
* We'll use the extension method to find all bar extensions of the selected bar in `barcode(VR)`

In [7]:
# select Witness bar of interest
W_bar = 2

2

<b> Run the extension method </b>

In [8]:
extension = run_extension_W_to_VR_bar(W = W, 
                                      W_bar = W_bar,
                                      C_VR = C_VR, 
                                      D_VR = D_VR, 
                                      dim = dim)

Dict{Any,Any} with 17 entries:
  "Ybar_rep_short_epsilon0" => Dict{Any,Any}(40=>[64],11=>[27],39=>[73],25=>[41…
  "aux_filt_cyclerep"       => Dict{Any,Any}(33=>[[35, 194], [51, 292], [101, 2…
  "Ybar_rep_tau"            => [75, 84, 79]
  "Ybar_rep_short"          => Dict{Any,Any}(32=>[54, 80, 64],26=>[11],43=>[69]…
  "selected_bar"            => 2
  "bar_extensions"          => Dict{Any,Any}(0.459276=>Dict{Any,Any}("offset"=>…
  "C_VR"                    => Dict{String,Any}("symmat"=>[44074 44058 … 0 0; 4…
  "C_W"                     => Dict{String,Any}("rv"=>Any[Int64[], [1, 2, 1, 3,…
  "C_auxiliary_filtration"  => Dict{String,Any}("rv"=>Any[Int64[], [1, 2, 1, 3,…
  "comparison"              => "W to VR"
  "cycle_extensions"        => Dict{Any,Any}(0.459276=>Dict{Any,Any}("offset"=>…
  "epsilon_0"               => 0.459276
  "p_Y"                     => [0.459276, 0.46891, 0.474519, 0.476031, 0.502393…
  "nontrivial_pY"           => [0.459276, 0.46891, 0.474519, 0.476031, 0.502393…
 

# 4. Explore the cycle extension & bar extension under fixed interval decomposition of `C_VR`
* As mentioned in "code_details.pdf", the `run_extension_W_to_VR_bar()` presents the component-wise cycle and bar extensions. This section illustrates the use of various functions to explore all cycle extension & bar extensions.
* For bar extensions, we only consider the result under a fixed interval decomposition of $PH_k(VR^{\bullet})$. 
* This section is organized as the following.  
    * (a) Plotting the parameters `p_Y`.   
    * (b) Interactively exploring the baseline and offset bar-extensions at various parameters.  
    * (c) Finding all cycle extensions and bar extensions (non-interactive). 
* Both subsection (b) and (c) illustrate how one may understand the bar-extensions. If your data contains large number of bars in the barcode of `C_auxiliary_filtration` and `C_VR`, then implementing the non-interactive method may take a while.

## 4(a) Plot all nontrivial p_Y values

In [9]:
plot_pY(extension)

<!DOCTYPE html>
 
 
 Plots.jl

## 4(b) Interactive exploration of baseline and offset bar extensions
* We use the function `return_extension_results_at_parameter()`, which is an interactive function that requires the user to select the following:
    * A parameter of `p_Y` 
    * Offset bar extensions
* The function shows the baseline bar-extension at selected parameter, along with the final bar-extension (baseline bar extension + selected offset bar extensions).
* The function returns a plot object that highlights the final bar extension

In [10]:
p = return_extension_results_at_parameter(extension)

*** Parameter key, value pair *** 
key: 1 parameter: 0.459276 
key: 2 parameter: 0.468910 
key: 3 parameter: 0.474519 
key: 4 parameter: 0.476031 
key: 5 parameter: 0.502393 
key: 6 parameter: 0.508495 
key: 7 parameter: 0.527879 
key: 8 parameter: 0.550888 
key: 9 parameter: 0.551174 
key: 10 parameter: 0.559679 
key: 11 parameter: 0.567726 
key: 12 parameter: 0.588657 
key: 13 parameter: 0.598662 
key: 14 parameter: 0.634918 
key: 15 parameter: 0.657946 
key: 16 parameter: 0.701574 
key: 17 parameter: 0.707338 
key: 18 parameter: 0.786701 



Select a key for parameter 1


Selected parameter: 0.45927577558008886

Baseline bars extension at selected parameter: [75, 84, 79]

*** Offset bar extensions at selected parameter *** 
key: 1 offset bar extension: [64]
key: 2 offset bar extension: [27]
key: 3 offset bar extension: [73]
key: 4 offset bar extension: [41]
key: 5 offset bar extension: [35]
key: 6 offset bar extension: [34, 28]
key: 7 offset bar extension: [38]
key: 8 offset bar extension: [40]
key: 9 offset bar extension: [10]
key: 10 offset bar extension: [39]
key: 11 offset bar extension: [28, 30]
key: 12 offset bar extension: [31]
key: 13 offset bar extension: [65]
key: 14 offset bar extension: [29]



Select keys for offset bar extensions. 
Leave blank to select none. 
To select multiple keys, separate keys with space. ex) 1 2 3 :  



Baseline bars extension at selected parameter: [75, 84, 79]


<!DOCTYPE html>
 
 
 Plots.jl

## 4(c) All cycle extensions and bar extensions 
* We use the function `find_CE_BE_at_param()` to find all cycle extensions and bar extensions at a specific parameter.
* Let `CE_param`, `BE_param` be the outputs of the function.  
    * `CE_param[i]` is the i-th cycle extension at given parameter.  
    * `BE_param[i]` is the bar extension of the corresponding cycle extension. 
* Note that one could use the function `find_CE_BE()` to find the cycle extensions and bar extensions at all parameters. For this particular example, the function takes a long time to complete.

In [11]:
# select parameter
param = extension["nontrivial_pY"][1]
CE_param, BE_param = find_CE_BE_at_param(extension, param);

<b> Plot cycle extensions </b>

In [12]:
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE_param))

number of cycle extensions at parameter 0.4593 : 16384

In [13]:
# plot 8 of the cycle extensions at selected parameter
ms = 3

p_objects = []
for i=0:7
    p = plot_cycle_square_torus(P, Q, cycle = CE_param[i], cycle_loc = "P", title = "cycle extension "*string(i),
                                P_markersize = 3, Q_markersize = 3; legend = false)

    push!(p_objects,p)
end

plot(p_objects..., layout = grid(2, 4), size = (700, 300))

<!DOCTYPE html>
 
 
 Plots.jl

Plot the <b>bar extensions</b> at given parameter.
* Select a cycle extension 
* Find and plot the corresponding bar extensions

In [14]:
# select parameter 
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE_param))

number of cycle extensions at parameter 0.4593 : 16384

In [15]:
# select cycle extension 
y= 1

1

In [16]:
# find the corresponding bar extension
be = BE_param[y]

# plot the bar extension
barcode_VR = barcode(C_VR, dim = dim)
p = plot_barcode(barcode_VR, title = "selected bar extension", lw = 2, selected_bars = be, epsilon= param, v_line = [param])
plot(p, size = (500, 300))

<!DOCTYPE html>
 
 
 Plots.jl

# 5. Explore the bar extension result under alternative interval decompositions of `C_VR`
* Up to this point, the bar extension result has been obtained for some fixed interval decomposition $\mathcal{D}:\mathbb{I}_{\text{BC}_k(Y^{\bullet})} \to H_k(Y^{\bullet})$ (where $Y^\bullet$ corresponds to the VR filtration). In particular, we used the default interval decomposition that is used by Eirene.
* In this section, we present various functions that allow us to find the full bar extensions under all possible interval decompositions of $PH_k(Y^{\bullet})$. The goal of this section is to explore $S(\tau, Y^{\bullet})$ from Algorithm 3 of paper. We'll refer to this set as <b>alternative bar extensions</b> since these arise from alternative choices of the interval decompositions.
* We present three different methods for exploring the collection of bar extensions $S(\tau, Y^{\bullet})$. The appropriate tool depends on the sizes of the barcodes of `C_auxiliary_filtration` and `C_VR`. 

1. Find all alternative bar extensions for all parameters.  
    * This is recommended for data with small barcodes. 
    * This finds the full $S(\tau, Y^{\bullet}) = \{ S^{\mathcal{D} \circ L^{-1}}_{[y]} | \ell \in p_Y, [y] \in \mathfrak{E}_{\ell}, L \in L_Y \}$ in Algorithm 3 of paper.
2. Find alternative bar extensions at specific parameters.  
    * This is recommended for data with medium size barcodes.
    * Given a parameter $\ell$, this method finds $S(\tau, Y^{\bullet}; \ell) = \{ S^{\mathcal{D} \circ L^{-1}}_{[y]} | [y] \in \mathfrak{E}_{\ell}, L \in L_Y \} $
3. Find alternative bar extensions of a specific bar extension.
    * This is recommended for data with large size barcodes.
    * Given a selected parameter $\ell$ and cycle extension $[y] \in \mathfrak{E}_{\ell}$, this method finds $\{S^{\mathcal{D} \circ L^{-1}}_{[y]} | L \in L_Y \}$. 
    
For this example, we'll implement method 3 due to the large number of bars `barcode(VR(Q))`. Depending on the selected parameter, the function `find_alternative_bar_extension()` can lead to an out of memory error. This will happen when a user selects a parameter at which there are numerous bars in the barcode. 

For example implementations of methods 1 and 2, see notebook `EXAMPLE_EXTENSION_VR_VR.ipynb`


Select a parameter and bar extension.

In [18]:
p = return_extension_results_at_parameter(extension)

*** Parameter key, value pair *** 
key: 1 parameter: 0.459276 
key: 2 parameter: 0.468910 
key: 3 parameter: 0.474519 
key: 4 parameter: 0.476031 
key: 5 parameter: 0.502393 
key: 6 parameter: 0.508495 
key: 7 parameter: 0.527879 
key: 8 parameter: 0.550888 
key: 9 parameter: 0.551174 
key: 10 parameter: 0.559679 
key: 11 parameter: 0.567726 
key: 12 parameter: 0.588657 
key: 13 parameter: 0.598662 
key: 14 parameter: 0.634918 
key: 15 parameter: 0.657946 
key: 16 parameter: 0.701574 
key: 17 parameter: 0.707338 
key: 18 parameter: 0.786701 



Select a key for parameter 13


Selected parameter: 0.5986623163978749

Baseline bars extension at selected parameter: [75, 84, 79]

*** Offset bar extensions at selected parameter *** 
key: 1 offset bar extension: [54, 80, 64]
key: 2 offset bar extension: [64]
key: 3 offset bar extension: [73]
key: 4 offset bar extension: [69]
key: 5 offset bar extension: [75, 73, 67]
key: 6 offset bar extension: [50]
key: 7 offset bar extension: [38]
key: 8 offset bar extension: [13, 70]
key: 9 offset bar extension: [79, 70]
key: 10 offset bar extension: [31]
key: 11 offset bar extension: [53]
key: 12 offset bar extension: [65]
key: 13 offset bar extension: [56]
key: 14 offset bar extension: [52]



Select keys for offset bar extensions. 
Leave blank to select none. 
To select multiple keys, separate keys with space. ex) 1 2 3 :  



Baseline bars extension at selected parameter: [75, 84, 79]


<!DOCTYPE html>
 
 
 Plots.jl

In [19]:
param = extension["nontrivial_pY"][13]
bar_ext = [75, 84, 79]

3-element Array{Int64,1}:
 75
 84
 79

In [22]:
# find alternative representations of the selected bar extension
alt_bar_ext = find_alternative_bar_extension(extension, param, bar_extension = bar_ext)

256-element Array{Array{Int64,1},1}:
 [75, 79, 84]
 [31, 75, 79, 84]
 [36, 75, 79, 84]
 [38, 75, 79, 84]
 [64, 75, 79, 84]
 [65, 75, 79, 84]
 [66, 75, 79, 84]
 [73, 75, 79, 84]
 [74, 75, 79, 84]
 [31, 36, 75, 79, 84]
 [31, 38, 75, 79, 84]
 [31, 64, 75, 79, 84]
 [31, 65, 75, 79, 84]
 ⋮
 [36, 38, 65, 66, 73, 74, 75, 79, 84]
 [36, 64, 65, 66, 73, 74, 75, 79, 84]
 [38, 64, 65, 66, 73, 74, 75, 79, 84]
 [31, 36, 38, 64, 65, 66, 73, 75, 79, 84]
 [31, 36, 38, 64, 65, 66, 74, 75, 79, 84]
 [31, 36, 38, 64, 65, 73, 74, 75, 79, 84]
 [31, 36, 38, 64, 66, 73, 74, 75, 79, 84]
 [31, 36, 38, 65, 66, 73, 74, 75, 79, 84]
 [31, 36, 64, 65, 66, 73, 74, 75, 79, 84]
 [31, 38, 64, 65, 66, 73, 74, 75, 79, 84]
 [36, 38, 64, 65, 66, 73, 74, 75, 79, 84]
 [31, 36, 38, 64, 65, 66, 73, 74, 75, 79, 84]

In [23]:
# plot one of the alternative bar extensions

# select an alternative bar extension
alt = alt_bar_ext[2]
barcode_Y = barcode(extension["C_VR"], dim = 1)
p =plot_barcode(barcode_Y, selected_bars = alt, lw = 3,
                    epsilon = param, v_line = [param],
                    title = "alternative intervals")
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl